# Lectura de Datos

In [21]:
import numpy as np
import os # provee una manera versátil de usar funcionalidades dependientes del sistema operativo
import gzip # lee y escribe archivos de formato gzip, comprimiendo o descomprimiendo automáticamente los datos para que parezcan un file object ordinario.

**Buffer:** Memoria de almacenamiento temporal de información que permite transferir los datos entre unidades funcionales con características de transferencia diferentes.

In [22]:
def load_set(ruta, tipo="train"):

  ruta_categorias = os.path.join(ruta,'%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

  with gzip.open(ruta_categorias, 'rb') as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8) # interpreta un buffer como una matriz unidimensional 

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas   

In [23]:
# Opción 1: cargarlo de drive
from google.colab import drive
drive.mount('/content/gdrive')

ruta = '/content/gdrive/MyDrive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Opción 2: traerlo de keras
%tensorflow_version 2.x
import tensorflow as tf
tf.keras.datasets.fashion_mnist.load_data()

### Reajustar tamaños de los datos

In [27]:
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984:]

X_train = np.reshape(X_train, (59904,28,28,1)) # Da una nueva forma a una matriz sin cambiar sus datos.
X_test = np.reshape(X_test, (9984,28,28,1))

In [28]:
print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.8.0


In [ ]:
tf.random.set_seed(200)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## Entrenamiento con CPU

In [ ]:
# CPU: Central Procesing Units (microprocesadores)

import timeit

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None
cpu_time =timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

Epoch 1/2
468/468 [==============================] - 775s 2s/step - loss: 0.6795 - accuracy: 0.7832 - val_loss: 0.3936 - val_accuracy: 0.8604
Epoch 2/2
398/468 [========================>.....] - ETA: 1:50 - loss: 0.3990 - accuracy: 0.8592

In [31]:
print('Tiempo de entrenamiento ' + str(cpu_time)+ ' segundos')

Tiempo de entrenamiento 1582.8167688390004 segundos


In [32]:
# Guardar predicciones
predicciones = model.predict(X_test)

In [33]:
# Guardar el Modelo
model.save('path_to_my_model.h5')

# Recrea exactamente el mismo modelo solo desde el archivo
modelo_CPU = tf.keras.models.load_model('path_to_my_model.h5')

modelo_CPU

## Entrenamiento con GPU

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

In [6]:
import os
import gzip 
import numpy as np 

def load_set(ruta, tipo="train"):

  ruta_categorias = os.path.join(ruta,'%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

  with gzip.open(ruta_categorias, 'rb') as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8) # interpreta un buffer como una matriz unidimensional 

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas   

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = '/content/gdrive/MyDrive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')

In [8]:
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984:]

X_train = np.reshape(X_train, (59904,28,28,1)) # Da una nueva forma a una matriz sin cambiar sus datos.
X_test = np.reshape(X_test, (9984,28,28,1))

In [ ]:
tf.random.set_seed(200)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [10]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')

print('GPU encontrada: {}'.format(nombre_gpu))

GPU encontrada: /device:GPU:0


In [11]:
# Graphics processing unit

import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None
gpu_time =timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')

Epoch 1/2
468/468 [==============================] - 37s 56ms/step - loss: 0.6844 - accuracy: 0.7813 - val_loss: 0.3982 - val_accuracy: 0.8673
Epoch 2/2
468/468 [==============================] - 26s 55ms/step - loss: 0.3925 - accuracy: 0.8625 - val_loss: 0.3271 - val_accuracy: 0.8878


In [12]:
print('Tiempo de entrenamiento ' + str(gpu_time)+ ' segundos')

Tiempo de entrenamiento 83.10359632400002 segundos


In [13]:
# Guardar predicciones
predicciones = model.predict(X_test)

In [14]:
# Guardar el Modelo
model.save('path_to_my_model.h5')

# Recrea exactamente el mismo modelo solo desde el archivo
modelo_GPU = tf.keras.models.load_model('path_to_my_model.h5')

modelo_GPU

## Entrenamiento con TPU

In [1]:
%tensorflow_version 2.x
import tensorflow as tf

In [2]:
import numpy as np
import os 
import gzip
from google.colab import drive

def load_set(ruta, tipo="train"):

  ruta_categorias = os.path.join(ruta,'%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)

  with gzip.open(ruta_categorias, 'rb') as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8) # interpreta un buffer como una matriz unidimensional 

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas   


In [3]:
drive.mount('/content/gdrive')

ruta = '/content/gdrive/MyDrive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984:]

X_train = np.reshape(X_train, (59904,28,28,1)) # Da una nueva forma a una matriz sin cambiar sus datos.
X_test = np.reshape(X_test, (9984,28,28,1))

In [6]:
# TPU: Tensor Processing Unit
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])  # Detectar TPU
  print('TPU encontrada ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU no encontrada!')

TPU encontrada  ['10.43.227.90:8470']


In [ ]:
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
tf.random.set_seed(200)

with tpu_strategy.scope():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  model.add(tf.keras.layers.Dropout(0.25))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(256))
  model.add(tf.keras.layers.Activation('elu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(10))
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()

  model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [11]:
import timeit

def entrenamiento_tpu():
  model.fit(X_train,Y_train, validation_data=(X_test,Y_test), batch_size=128, epochs=2, verbose=1)
  
  return None

tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')

Epoch 1/2
468/468 [==============================] - 17s 23ms/step - loss: 0.7230 - accuracy: 0.7742 - val_loss: 0.3781 - val_accuracy: 0.8682
Epoch 2/2
468/468 [==============================] - 9s 20ms/step - loss: 0.4028 - accuracy: 0.8577 - val_loss: 0.3465 - val_accuracy: 0.8851


In [12]:
print('Tiempo de entrenamiento ' + str(tpu_time)+ ' segundos')

Tiempo de entrenamiento 27.693097442999715 segundos


In [14]:
history = model.fit(X_train, Y_train, batch_size=128, epochs=2)

Epoch 1/2
468/468 [==============================] - 8s 17ms/step - loss: 0.3267 - accuracy: 0.8822
Epoch 2/2
468/468 [==============================] - 8s 18ms/step - loss: 0.2842 - accuracy: 0.8960


In [17]:
# Guardar predicciones
predicciones = model.predict(X_test)

In [20]:
# Guardar el Modelo
model.save('path_to_my_model.h5')

# Recrea exactamente el mismo modelo solo desde el archivo
modelo_TPU = tf.keras.models.load_model('path_to_my_model.h5')

In [21]:
modelo_TPU